In [10]:
import os
from pathlib import Path
from dotenv import load_dotenv

PROJECT_ROOT = Path.cwd().parent   # porque estás en /experiments
load_dotenv(PROJECT_ROOT / ".env")

chroma_env = os.getenv("CHROMA_DIR", "vector_store/chroma")
CHROMA_PATH = (PROJECT_ROOT / chroma_env).resolve()  # <-- clave

print("PROJECT_ROOT:", PROJECT_ROOT)
print("CHROMA_DIR del .env:", chroma_env)
print("CHROMA_PATH final:", CHROMA_PATH)
print("Existe?:", CHROMA_PATH.exists())


PROJECT_ROOT: c:\Users\natalia.antepazo\OneDrive - Grupo Gimeno\Repos\medical_bot
CHROMA_DIR del .env: vector_store/chroma
CHROMA_PATH final: C:\Users\natalia.antepazo\OneDrive - Grupo Gimeno\Repos\medical_bot\vector_store\chroma
Existe?: True


In [11]:
import chromadb

client = chromadb.PersistentClient(path=str(CHROMA_PATH))
print([c.name for c in client.list_collections()])


['medical_kb']


count: 4


In [13]:
import chromadb
from pathlib import Path
import os
from dotenv import load_dotenv

PROJECT_ROOT = Path.cwd().parent  # estás en experiments/
load_dotenv(PROJECT_ROOT / ".env")

chroma_env = os.getenv("CHROMA_DIR", "vector_store/chroma")
CHROMA_PATH = (PROJECT_ROOT / chroma_env).resolve()

client = chromadb.PersistentClient(path=str(CHROMA_PATH))

cols = client.list_collections()
print("CHROMA_PATH:", CHROMA_PATH)
print("Colecciones encontradas:", [c.name for c in cols])

for c in cols:
    col = client.get_collection(c.name)
    print(f"- {c.name}: {col.count()}")


CHROMA_PATH: C:\Users\natalia.antepazo\OneDrive - Grupo Gimeno\Repos\medical_bot\vector_store\chroma
Colecciones encontradas: ['medical_kb', 'med_kb']
- medical_kb: 4
- med_kb: 0


# 1) Celda: abrir Chroma y la colección

In [15]:
COLLECTION_NAME = "medical_kb"  # el que pusiste en build_kb.py
col = client.get_or_create_collection(COLLECTION_NAME)
print("count:", col.count())

count: 4


# 2) Celda: ver “filas” reales (IDs, metadatos, documentos)

In [16]:
sample = col.get(limit=5, include=["documents", "metadatas"])
len(sample["ids"]), sample.keys()


(4,
 dict_keys(['ids', 'embeddings', 'documents', 'uris', 'included', 'data', 'metadatas']))

In [18]:
for i in range(len(sample["ids"])):
    print("\n--- ITEM", i, "---")
    print("id:", sample["ids"][i])
    print("meta:", sample["metadatas"][i])
    print("doc:", sample["documents"][i][:400], "...")



--- ITEM 0 ---
id: 2984db474a0075e823914c36b18d88d748b12fe0
meta: {'source': 'docs\\sources\\clinic_admin.md', 'chunk_index': 0, 'doc_type': 'md'}
doc: # Información administrativa de la clínica

## Horarios
La clínica atiende de lunes a viernes de 9:00 a 13:30 y de 16:00 a 19:00.

## Dirección
La clínica se encuentra en Valencia ciudad. La dirección es calle Mestre Sosa 37

## Solicitud de cita
Las citas pueden solicitarse por teléfono en el 666666666 o a través del formulario web.

## Urgencias
Si el paciente presenta síntomas graves, debe acud ...

--- ITEM 1 ---
id: bda909f04bf16a4a35e20c397b0717d0f0807d7f
meta: {'doc_type': 'md', 'chunk_index': 0, 'source': 'docs\\sources\\red_flag.md'}
doc: # Información general sobre urología

La urología es la especialidad médica que se encarga del estudio y tratamiento del aparato urinario y del aparato reproductor masculino.

## ¿Cuándo acudir al urólogo?
Es habitual consultar con un especialista en urología ante síntomas como:
- Molestias u

# 3) Celda: cuántos chunks por fichero (muy útil)

In [19]:
from collections import Counter

all_meta = col.get(include=["metadatas"])["metadatas"]
sources = [m.get("source", "unknown") for m in all_meta]
counts = Counter(sources)

for src, n in counts.most_common():
    print(f"{n:>4}  {src}")


   1  docs\sources\clinic_admin.md
   1  docs\sources\red_flag.md
   1  docs\sources\services.md
   1  docs\sources\urology_general.md


# 4) Celda: hacer una query (la parte “vectorial”)

In [20]:
query = "¿Cuál es el horario de la clínica?"
res = col.query(
    query_texts=[query],
    n_results=5,
    include=["documents", "metadatas", "distances"]
)

print("QUERY:", query)
for i in range(len(res["documents"][0])):
    print("\n--- RESULT", i+1, "---")
    print("distance:", res["distances"][0][i])
    print("source:", res["metadatas"][0][i].get("source"))
    print(res["documents"][0][i][:400], "...")


ConnectError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000) in query.

Ese error en la query no viene de Chroma “buscando”, viene de que Chroma necesita convertir tu pregunta a embedding y, al hacerlo, está intentando llamar a un servicio de embeddings (o descargar algo) y te choca con el SSL corporativo (certificado self-signed).

La clave:

col.query(query_texts=[...]) ⇒ Chroma tiene que embeddar el texto de la query ⇒ puede disparar red ⇒ SSL error.

Si en cambio le pasas ya el embedding, Chroma no hace ninguna llamada externa.

Vamos a hacerlo “pro” desde notebook: calcular tú el embedding con OpenAI

In [ ]:
import os
from openai import OpenAI

EMBED_MODEL = os.getenv("MODEL_EMBED", "text-embedding-3-small")
client_oa = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

question = "¿Cuál es el horario de la clínica?"
q_emb = client_oa.embeddings.create(model=EMBED_MODEL, input=question).data[0].embedding
len(q_emb)



1536

In [23]:
q_emb

[-0.0356510691344738,
 0.0333157517015934,
 0.02950211614370346,
 -0.004105550702661276,
 -0.040171727538108826,
 0.0692024976015091,
 0.03271585330367088,
 0.013401288539171219,
 -0.01513670664280653,
 -0.032801553606987,
 -0.015383093617856503,
 -0.020332250744104385,
 0.020953573286533356,
 -0.020010877400636673,
 -0.035565368831157684,
 0.02573133073747158,
 -0.022560441866517067,
 0.011901543475687504,
 -0.018018359318375587,
 0.028345171362161636,
 -0.02298893965780735,
 0.0013926199171692133,
 0.040428824722766876,
 -0.03528684377670288,
 -0.03140893578529358,
 -0.006422120146453381,
 -0.0022107840050011873,
 -0.027959521859884262,
 0.021392783150076866,
 0.0072255549021065235,
 -0.01054106093943119,
 -0.04045025259256363,
 0.02603127993643284,
 0.0356510691344738,
 -0.04287126660346985,
 0.026224102824926376,
 0.004105550702661276,
 0.02521713264286518,
 0.005527629517018795,
 -0.01462250854820013,
 -0.03712938725948334,
 -0.003998425789177418,
 -0.026674026623368263,
 -0.04336

In [24]:
COLLECTION_NAME = "medical_kb"  
col = client.get_collection(COLLECTION_NAME)

res = col.query(
    query_embeddings=[q_emb],
    n_results=5,
    include=["documents", "metadatas", "distances"],
)

print("Q:", question)
for i in range(len(res["documents"][0])):
    print("\n---", i+1, "---")
    print("dist:", res["distances"][0][i])
    print("src :", res["metadatas"][0][i].get("source"))
    print(res["documents"][0][i][:300], "...")


Q: ¿Cuál es el horario de la clínica?

--- 1 ---
dist: 0.3084239959716797
src : docs\sources\clinic_admin.md
# Información administrativa de la clínica

## Horarios
La clínica atiende de lunes a viernes de 9:00 a 13:30 y de 16:00 a 19:00.

## Dirección
La clínica se encuentra en Valencia ciudad. La dirección es calle Mestre Sosa 37

## Solicitud de cita
Las citas pueden solicitarse por teléfono en el 66666 ...

--- 2 ---
dist: 0.5360060334205627
src : docs\sources\services.md
# Servicios de urología

La clínica ofrece atención especializada en urología para adultos.

## Servicios habituales
- Problemas de próstata
- Litiasis urinaria (cálculos renales)
- Infecciones urinarias
- Disfunción eréctil
- Vasectomía
- Fertilidad masculina

Toda la información proporcionada es d ...

--- 3 ---
dist: 0.6789499521255493
src : docs\sources\urology_general.md
# Información general sobre urología

La urología es la especialidad médica que se encarga del estudio y tratamiento del aparato urinario y 

# 5) Celda: probar varias queries típicas del bot

In [26]:
queries = [
    "¿Cómo puedo pedir cita?",
    "¿Qué servicios ofrecéis?",
    "¿Qué hace un urólogo?",
    "Tengo fiebre y no puedo orinar, ¿qué hago?"
]

for q in queries:
    q_emb = client_oa.embeddings.create(model=EMBED_MODEL, input=q).data[0].embedding

    res = col.query(
        query_embeddings=[q_emb],
        n_results=2,
        include=["metadatas", "documents", "distances"]
    )

    print("\n==============================")
    print("Q:", q)
    for i in range(len(res["documents"][0])):
        print(f"- {i+1} dist={res['distances'][0][i]:.4f} src={res['metadatas'][0][i].get('source')}")




Q: ¿Cómo puedo pedir cita?
- 1 dist=0.5495 src=docs\sources\clinic_admin.md
- 2 dist=0.6652 src=docs\sources\services.md

Q: ¿Qué servicios ofrecéis?
- 1 dist=0.5536 src=docs\sources\services.md
- 2 dist=0.6541 src=docs\sources\clinic_admin.md

Q: ¿Qué hace un urólogo?
- 1 dist=0.3243 src=docs\sources\red_flag.md
- 2 dist=0.3244 src=docs\sources\urology_general.md

Q: Tengo fiebre y no puedo orinar, ¿qué hago?
- 1 dist=0.6096 src=docs\sources\red_flag.md
- 2 dist=0.6097 src=docs\sources\urology_general.md
